## ML Model Training

In [42]:
import pandas as pd 
import numpy as np
import joblib

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [ ]:
df = pd.read_csv('D:\m_bi_data1\sensor_data.csv')

In [44]:
df

,time,torque,current,temp,rotations,vibration,problem_id,category,rul
0,0.00000,53.847860,8.681045,28.899380,1023.086320,0.975967,N0_0,normal,NaN
1,0.01001,49.405651,8.823417,25.925456,1007.463987,0.524702,N0_0,normal,NaN
2,0.02002,49.644757,10.580681,31.347094,1044.937537,0.756580,N0_0,normal,NaN
3,0.03003,50.655802,9.212178,30.292826,1017.587298,0.933422,N0_0,normal,NaN
4,0.04004,46.624697,9.426600,31.084558,1011.478908,1.322530,N0_0,normal,NaN
...,...,...,...,...,...,...,...,...,...
3439995,9.95996,47.538846,16.682748,48.248493,870.190664,3.883119,F7_79,pre_fail,0.0
3439996,9.96997,54.574131,19.106418,44.858397,850.937128,3.507954,F7_79,pre_fail,0.0
3439997,9.97998,50.096030,17.063657,44.421853,851.303169,2.994856,F7_79,pre_fail,0.0
3439998,9.98999,46.862231,16.686221,44.142924,836.278639,3.694505,F7_79,pre_fail,0.0


In [45]:
df["problem_type"] = df["problem_id"].apply(lambda x: x.split("_")[0])


In [ ]:
df["problem_type"].unique()


<StringArray>
[ 'N0',  'S1',  'S2',  'S3',  'S4',  'S5',  'S6',  'S7',  'S8',  'S9', 'S10',
 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18',  'D1',  'D2',  'D3',
  'D4',  'D5',  'D6',  'D7',  'D8',  'D9', 'D10', 'D11', 'D12', 'D13', 'D14',
  'M1',  'M2',  'M3',  'F1',  'F2',  'F3',  'F4',  'F5',  'F6',  'F7']
Length: 43, dtype: str


## Feature Engineering

In [ ]:
features = []

grouped = df.groupby("problem_id")

for pid, group in grouped:
    
    feat = {}
    feat["problem_id"] = pid
    feat["problem_type"] = group["problem_type"].iloc[0]
    
    for col in ["torque","current","temp","rotations","vibration"]:
        
        values = group[col].values
        
        feat[f"{col}_mean"] = np.mean(values)
        feat[f"{col}_std"] = np.std(values)
        feat[f"{col}_max"] = np.max(values)
        feat[f"{col}_min"] = np.min(values)
        
        
        x = np.arange(len(values))
        slope = np.polyfit(x, values, 1)[0]
        feat[f"{col}_trend"] = slope
        feat[f"{col}_energy"] = np.sum(values**2)
    
    features.append(feat)

features_df = pd.DataFrame(features)
features_df.head()

  problem_id problem_type  torque_mean  torque_std  torque_max  torque_min  \
0      D10_0          D10    50.657986    3.452815   62.314742   39.668724   
1      D10_1          D10    48.445707    3.449452   58.937546   38.291089   
2     D10_10          D10    50.300915    3.560129   61.162757   37.734942   
3     D10_11          D10    49.033251    3.512951   62.022280   38.438362   
4     D10_12          D10    48.577870    3.458028   57.575413   36.946514   

   torque_trend  torque_energy  current_mean  current_std  ...  rotations_max  \
0     -0.004190   2.578153e+06      9.020206     1.350491  ...    1065.889346   
1     -0.004598   2.358885e+06      9.071091     1.456998  ...    1152.708430   
2     -0.004641   2.542857e+06      8.974506     1.305796  ...    1129.350091   
3     -0.004956   2.416601e+06      9.178841     1.271494  ...    1093.388299   
4     -0.004405   2.371767e+06      8.545536     1.344595  ...    1106.492071   

   rotations_min  rotations_trend  rotations

In [48]:
X = features_df.drop(["problem_id","problem_type"], axis=1)

le = LabelEncoder()
y = le.fit_transform(features_df["problem_type"])

In [49]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [50]:

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [51]:
model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train_scaled, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",200
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [ ]:
pred = model.predict(X_test_scaled)
acc = accuracy_score(y_test, pred)

 

Accuracy: 0.9418604651162791


In [ ]:
acc

In [53]:
probs = model.predict_proba(X_test_scaled)


In [54]:
probs.shape

(688, 43)

In [55]:
sample_probs = probs[0]

for cls, p in zip(le.classes_, sample_probs):
    print(cls, round(p,3))


D1 0.0
D10 0.0
D11 0.0
D12 0.0
D13 0.0
D14 0.0
D2 0.0
D3 0.0
D4 0.0
D5 0.0
D6 0.98
D7 0.0
D8 0.0
D9 0.0
F1 0.0
F2 0.005
F3 0.0
F4 0.0
F5 0.0
F6 0.0
F7 0.0
M1 0.0
M2 0.0
M3 0.0
N0 0.0
S1 0.0
S10 0.0
S11 0.0
S12 0.0
S13 0.0
S14 0.0
S15 0.0
S16 0.0
S17 0.0
S18 0.015
S2 0.0
S3 0.0
S4 0.0
S5 0.0
S6 0.0
S7 0.0
S8 0.0
S9 0.0


In [56]:
joblib.dump(model, "model.pkl")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(le, "label_encoder.pkl")

['label_encoder.pkl']